# Enunciado

Modele e descreva um sistema de recomendação de um livro a partir do dataset fornecido.

Realize a exploração das tabelas fornecidas com o intuito de conhecer os dados e criar um sistema de recomendação mais assertivo.



1. A exploração das três tabelas é um item obrigatório nesta avaliação (exploração livre);
2. Descreva detalhadamente como seria o modelo de recomendação, quais campos seriam utilizados e forneça os motivos para cada escolha;

3. Descreva qual algoritmo de Aprendizado de Máquina você utilizaria e o porquê;

4. Como seria realizada a validação do algoritmo?


**OBS. IMPORTANTE: Para os itens 2, 3 e 4 não é necessario gerar código, mas as justificativas devem estar muito bem embasadas na exploração dos dados realizada no item 1. Para o item será necessário o desenvolvimento de código.**

In [ ]:
# Importação dentro de Google Colab
# Caso de algum erro ao baixar o arquivo, remova o comentário de outra fonte de dados do gdown
!gdown 1OVfOVHmakYE8XmjJFVruUnx3QJOPTLGk
#!gdown 1-cVAXcv8wQOEVQ6kLEb_NNUP6hk7kTX2
#!gdown 1_BJJzuD2R-lxryyV124QkwDK9CwIpuYm

import zipfile
zip_path = '/content/archive.zip'
extract_path = '/content'

# Abre o arquivo zip e extrai os arquivos para o diretório especificado
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Downloading...
From: https://drive.google.com/uc?id=1OVfOVHmakYE8XmjJFVruUnx3QJOPTLGk
To: /content/archive.zip
100% 24.9M/24.9M [00:00<00:00, 86.8MB/s]


In [ ]:
# Importação de bibliotecas

import numpy as np
import pandas as pd
import sklearn.metrics.pairwise as pw

In [ ]:
# Leitura de arquivos

df_books = pd.read_csv("/content/Books.csv")
df_ratings = pd.read_csv("/content/Ratings.csv")
df_users = pd.read_csv("/content/Users.csv")

<ipython-input-5-80e675267496>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv("/content/Books.csv")



## Tabelas

**Users**:

Contém os usuários. Observe que os IDs dos usuários (ID do usuário) foram anonimizados e mapeados para inteiros. Dados demográficos são fornecidos (Localização, Idade) se disponíveis. Caso contrário, esses campos contêm valores NULL.

**Books**:

Os livros são identificados por seus respectivos ISBN. ISBNs inválidos já foram removidos do conjunto de dados. Além disso, algumas informações baseadas em conteúdo são fornecidas (Título do livro, Autor do livro, Ano de Publicação, Editora), obtidas do Amazon Web Services. Observe que, no caso de vários autores, apenas o primeiro é fornecido. URLs que apontam para imagens de capa também são fornecidos, aparecendo em três diferentes sabores (Image-URL-S, Image-URL-M, Image-URL-L), ou seja, pequeno, médio e grande. Esses URLs apontam para o site da Amazon.

**Ratings**:

Contém informações de classificação de livros. As classificações (Classificação do livro) são explícitas, expressas em uma escala de 1 a 10 (valores mais altos denotando maior apreciação), ou implícitas, expressas por 0.



# Exploração (item 1)

In [ ]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
df_books.shape

(271360, 8)

In [ ]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
df_ratings.shape

(1149780, 3)

In [ ]:
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
df_users.shape

(278858, 3)

In [ ]:
print(df_books.isnull().sum())
print("\n")
print(df_ratings.isnull().sum())
print("\n")
print(df_users.isnull().sum())

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


User-ID          0
Location         0
Age         110762
dtype: int64


# Modelagem do Sistema de Recomendação

## Item 2

Com base na exploração dos dados disponíveis, uma ideia adequada de sistema de recomendação seria baseada num **sistema de filtragem colaborativa**.

Isso pode ser justificado pelo fato da modelagem de um sistema de recomendação por conteúdo não ser efetiva nesse caso, uma vez que:


*   Não é possível aplicar TF-IDF (Term Frequence - Inverse Document Frequence) adequadamente, já que não existe uma coluna referente a sinopses dos livros e aplicar essa ferramenta estatística com base apenas na coluna de títulos não seria eficiente, havendo a chance das recomendações girarem em torno apenas de livros de uma mesma série

*   Não há uma coluna contendo os gêneros de cada livro


Contudo, em *Ratings.csv*, observa-se vários dados de avaliações de livros por usuários (identificados por id), o que é muito útil para o modelo de filtragem colaborativa. Sabendo-se, ainda, que vários usuários avaliaram mais de um livro, é possível modelar o sistema de forma que um usuário que atribuiu uma nota alta a um determinado livro receba a recomendação de um outro livro bem avaliado por um usuário que também atribuiu uma nota alta a tal livro.

Também vale ressaltar que existe, na base de dados, colunas irrelevantes para a modelagem do sistema de recomendação, como as referentes a imagens de capa dos livros ("Image-URL-S",	"Image-URL-M" e	"Image-URL-L") em *Books.csv*, e muitos dados nulos na coluna "Age" (referente à idade dos usuários) em *Users.csv*. Logo, é viável deletá-las na etapa de pré-processamento de dados.

Pensando na construção do sistema de fato, cabe pensar nas seguintes etapas:


*   Limitar a base de dados a usuários que fizeram uma quantidade mínima de avaliações

*   Calcular coeficiente de correlação entre usuários de acordo com as avaliações que estes deram aos livros

*   Dado um usuário X, definir os 10 outros usuários com maior índice de correlação em relação a ele

*   Selecionar, entre os livros lidos e mais bem-avaliados pelo usuário mais correlacionado ao usuário X, os livros não lidos pelo usuário X e recomendá-los para ele

In [ ]:
# Limitando o dataset apenas a usuários que fizeram 200 ou mais avaliações

numero_de_avaliacoes_por_id = df_ratings['User-ID'].value_counts()

ids_para_manter = numero_de_avaliacoes_por_id[numero_de_avaliacoes_por_id >= 200].index

df_ratings = df_ratings[df_ratings['User-ID'].isin(ids_para_manter)]

df_ratings.head()

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [ ]:
# Fundindo datasets de avaliações e livros

df = df_books.merge(df_ratings, on='ISBN')
df.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis = 1, inplace=True) # Excluindo colunas inúteis
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,11676,8
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,85526,0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,96054,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,177458,0
4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,110912,10


In [ ]:
# Criando matriz relacionando livros e usuários por avaliação atribuida ao livro pelo usuário

matriz_livros = pd.pivot_table(df, index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)

matriz_livros.head()

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Always Have Popsicles,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apple Magic (The Collector's series),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Beyond IBM: Leadership Marketing and Finance for the 1990s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Clifford Visita El Hospital (Clifford El Gran Perro Colorado),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Definindo coeficientes de correlação entre usuários com base nas avaliações feitas por eles
# Método escolhido: coeficiente de Pearson (quanto mais próximo de 1, mais semelhante é o gosto entre os usuários em quesão)

matriz_correlacao = matriz_livros.corr(method='pearson')
matriz_correlacao

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
User-ID,,,,,,,,,,,,,,,,,,,,,
254,1.000000,-0.000609,-0.000407,0.018337,0.013841,0.010140,-0.000545,-0.000643,-0.000506,0.038364,...,-0.000431,0.009488,-0.000554,0.023980,-0.000302,0.042113,-0.000529,-0.000255,-0.000241,-0.000438
2276,-0.000609,1.000000,0.012067,-0.000642,-0.000822,-0.000815,-0.001049,0.004670,-0.000974,0.002064,...,0.014837,-0.001235,-0.001066,-0.000571,-0.000581,-0.001000,-0.001018,-0.000491,0.023789,-0.000844
2766,-0.000407,0.012067,1.000000,-0.000429,0.006687,-0.000545,0.026592,-0.000828,0.021249,-0.000824,...,0.021763,-0.000826,0.014943,-0.000382,-0.000388,0.005586,0.019830,0.016627,-0.000310,0.006620
2977,0.018337,-0.000642,-0.000429,1.000000,-0.000450,-0.000446,0.006795,-0.000677,0.010860,0.016548,...,-0.000454,-0.000675,0.011126,-0.000312,-0.000318,-0.000547,-0.000557,-0.000269,-0.000254,-0.000461
3363,0.013841,-0.000822,0.006687,-0.000450,1.000000,-0.000571,0.026635,-0.000868,-0.000683,-0.000864,...,-0.000582,-0.000866,-0.000747,-0.000400,0.019449,-0.000701,-0.000714,-0.000344,-0.000325,-0.000591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275970,0.042113,-0.001000,0.005586,-0.000547,-0.000701,-0.000695,0.006202,-0.001056,-0.000830,0.016633,...,0.008927,-0.001053,-0.000909,0.012886,-0.000495,1.000000,0.012688,-0.000419,-0.000395,-0.000719
277427,-0.000529,-0.001018,0.019830,-0.000557,-0.000714,-0.000707,0.006847,-0.001075,0.008673,0.021517,...,0.008759,-0.001072,0.014483,0.036961,-0.000504,0.012688,1.000000,-0.000426,-0.000403,-0.000732
277478,-0.000255,-0.000491,0.016627,-0.000269,-0.000344,-0.000341,-0.000439,-0.000518,-0.000408,-0.000516,...,-0.000347,-0.000517,-0.000446,-0.000239,-0.000243,-0.000419,-0.000426,1.000000,-0.000194,-0.000353


In [ ]:
# Definindo função para receber recomendações com base no id de usuário

def receber_recomendacoes(user_id):

  coeficientes_positivos = [] # Lista para receber coeficientes positivos entre o usuário definido e os demais

  coeficientes = matriz_correlacao[user_id].values

  # Loop para acrescentar coeficientes positivos à lista
  for c in coeficientes:
    if c > 0:
      coeficientes_positivos.append(c)

  coeficientes_positivos.sort(reverse=True)

  top_10_coeficientes = coeficientes_positivos[1:11] # Seleção dos 10 maiores coeficientes entre o usuário definido e os demais

  outro_id = matriz_correlacao.loc[matriz_correlacao[user_id].values == top_10_coeficientes[0]].index # Salva o ID do usuário que possui o maior coeficiente de correlação com o usuário definido

  lista_recomendacoes = df.loc[df['User-ID'] == outro_id[0]].sort_values(by='Book-Rating', ascending=False)['Book-Title'].values[0:10] # Salva os 10 livros mais bem avaliados pelo usuário salvo em "outro_id"

  lista_lidos = df.loc[df['User-ID'] == user_id]['Book-Title'].values

  lista_final = np.setdiff1d(lista_recomendacoes, lista_lidos) # Remove livros já lidos pelo usuário definido da lista de livros mais bem avaliados pelo usuário salvo em "outro_id"

  return lista_final


In [ ]:
receber_recomendacoes(277639)

<class 'numpy.ndarray'>


array(['A RIVER RUNS THROUGH IT AND OTHER STORIES : AND OTHER STORIES',
       'Never Say Die',
       "New Believer's Bible New Testament: New Living Translation (New Living Translation)",
       'Ogre, Ogre',
       "Shadow Watch (Tom Clancy's Power Plays (Paperback))",
       'Step by Step Book About Dwarf Hamsters',
       'Tales of Mystery and Terror', 'The Tao of Pooh',
       "The Truth About Witchcraft Today (Llewellyn's New Age)"],
      dtype=object)

In [ ]:
# @title Teste de sistema { run: "auto" }
# @markdown Digite o seu id de usuário para obter recomendações de livros:
id_usuario = 254 # @param {type:"integer"}

print('\nPessoas como você gostaram de ler: \n')
for i in receber_recomendacoes(id_usuario):
  print(i)



Pessoas como você gostaram de ler: 

A Kiss for Little Bear
All Creatures Great and Small
Dr. Seuss's A B C (I Can Read It All by Myself Beginner Books)
Little Bear's Friend
Little Bear's Visit (I Can Read Book 1)
The First Eagle
The Foot Book (Bright and Early Books for Beginning Beginners)
The Last Enchantment : Book Three of the Arthurian Saga (Arthurian Saga, 3)
The Tao of Pooh
Turtle Moon


## Item 3


Com base na modelagem pensada anteriormente, não é necessário aplicar um algoritmo de aprendizagem de máquina para esse sistema de recomendação.

## Item 4


A validação do sistema de recomendação poderia ser feita através dos próprios usuários. Isto é, ao ler determinado livro e receber recomendações relacionadas a tal livro, o usuário pode ter a opção de, após ler os livros recomendados, avaliar tais recomendações. Dessa forma, há um feedback do quão adequadas determinadas recomendações foram e é possível ajustar o modelo conforme ocorre a utilização deste.